Instalação de dependências:

In [ ]:
# !pip install pandas
# !pip install python-dotenv
# !pip install openai

In [2]:
# Importação das bibliotecas
from openai import OpenAI
from dotenv import load_dotenv

# Carregar variáveis de ambiente